In [1]:
#neki chatgpt
'''def find_best_node(graph, NB, B):
    """
    Pronalaženje čvora koji pokriva najviše neprekrivenih čvorova.
    """
    max_covered = -1
    best_node = None

    for node in NB:
        # Prebrojavanje neprekrivenih suseda trenutnog čvora
        uncovered_neighbors = sum(1 for neighbor in graph[node] if neighbor not in B)
        if uncovered_neighbors > max_covered:
            max_covered = uncovered_neighbors
            best_node = node

    return best_node


def generate_random_burning_sequence_with_greedy(graph):
    """
    Generisanje sekvence sagorevanja sa smanjenom nasumičnošću pomoću grešnog pristupa.
    """
    S = []  # sekvenca zapaljenih čvorova
    B = set()  # skup zapaljenih čvorova
    NB = set(graph.keys())  # skup nezapaljenih čvorova, inicijalno su to svi čvorovi grafa

    # Prvi čvor se bira nasumično
    current_node = random.choice(list(NB))
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)

    while len(B) < len(graph):
        # Sagorevanje suseda trenutnog čvora
        neighbors_to_burn = [neighbor for neighbor in graph[current_node] if neighbor in NB]
        for neighbor in neighbors_to_burn:
            B.add(neighbor)
            NB.remove(neighbor)

        # Pronalaženje najboljeg sledećeg čvora
        best_node = find_best_node(graph, NB, B)

        if best_node is not None:
            # Dodavanje najboljeg čvora u sekvencu
            S.append(best_node)
            B.add(best_node)
            NB.remove(best_node)
            current_node = best_node  # Ažuriramo trenutni čvor
        else:
            # Ako nema "najboljeg" čvora, biramo nasumičan čvor
            if NB:
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
                current_node = random_node
                # Ažuriramo trenutni čvor
            else:
                break
    return S


def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    """
    Generisanje više sekvenci sagorevanja sa smanjenom nasumičnošću.
    """
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence_with_greedy(graph)
        if sequence not in sequences and valid_burning_sequence(sequence, len(graph), all_pairs_distance_matrix(graph)):
            sequences.append(sequence)
    return sequences

ModuleNotFoundError: No module named 'gurobipy'

In [ ]:
#beetweness
'''def bfs_shortest_paths(graph, start):
    dist = {node: float('inf') for node in graph} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graph}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graph[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_betweenness(graph):
    betweenness = {node: 0 for node in graph}
    
    for start in graph:
        dist, paths = bfs_shortest_paths(graph, start)
        
        for node in graph:
            if node == start:
                continue
            
            for neighbor in paths[node]:
                if neighbor != start:
                    betweenness[neighbor] += 1
    
    return betweenness

def betweenness_centralnost(node, betweenness):
    return betweenness.get(node, 0) 
def choose_initial_node(graph, betweenness):
    """
    Kombinuje heuristički i nasumični izbor početnog čvora.
    """
    top_k = int(len(graph)/3)
    # Sortiraj čvorove po betweenness centralnosti
    sorted_nodes = sorted(graph.keys(), key=lambda node: betweenness_centralnost(node, betweenness), reverse=True)
    
    # Uzmi top_k čvorova (ili manje ako ih nema dovoljno)
    top_candidates = sorted_nodes[:top_k]
    
    # Nasumično izaberi jednog od kandidata
    return random.choice(top_candidates)

def greedy_burning(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    precomputed_betweenness = calculate_betweenness(graph)

    current_node = choose_initial_node(graph, precomputed_betweenness)

    
    
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors>max_burned:
                        max_burned = notburned_neighbors
                        
            for node in NB:
                if node not in FB2:
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors == max_burned:
                        candidates_for_best.append(node) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            
            
            if len(candidates_for_best)>1:# izbor na osnovu betweenness centralnosti
                 best_node = max(candidates_for_best, key=lambda node: betweenness_centralnost(node, calculate_betweenness(graph)))
            elif len(candidates_for_best)==1:
                best_node = candidates_for_best[0]
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S


def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    """
    Generisanje više sekvenci sagorevanja sa smanjenom nasumičnošću.
    """
    sequences = []
    for _ in range(num_sequences):
        sequence = greedy_burning(graph)
        if sequence not in sequences and valid_burning_sequence(sequence, len(graph), all_pairs_distance_matrix(graph)):
            sequences.append(sequence)
    return sequences
burning_sequences2 = generate_multiple_random_burning_sequences_with_greedy(graph, 50)
for seq in burning_sequences2:
    print(seq, "len:", len(seq))'''
        

In [ ]:
#po susjedima
'''def greedy_burning(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    
    neighbors_count = calculate_neighbors(graph)

    current_node = choose_initial_node(graph, neighbors_count)
    
    
        
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors>max_burned:
                        max_burned = notburned_neighbors
                        
            for node in NB:
                if node not in FB2:
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors == max_burned:
                        candidates_for_best.append(node) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            
            
            if len(candidates_for_best)>1:# izbor na osnovu betweenness centralnosti
                 best_node = random.choice(candidates_for_best)
            elif len(candidates_for_best)==1:
                best_node = candidates_for_best[0]
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    """
    Generisanje više sekvenci sagorevanja sa smanjenom nasumičnošću.
    """
    sequences = []
    for _ in range(num_sequences):
        sequence = greedy_burning(graph)
        sequences.append(sequence)
    return sequences



burning_sequences = generate_multiple_random_burning_sequences_with_greedy(graph, 50)
for seq in burning_sequences:
    print(seq, "len:", len(seq))      '''  

In [ ]:
#ucitavanje

# Učitavanje grafa
#filename = "grafovi/karate.mtx"
#filename = "grafovi/chesapeake.mtx"
#filename = "grafovi/dolphins.mtx"
#filename = "grafovi/rt-retweet.mtx"
#filename = "grafovi/polbooks.mtx"
#filename = "grafovi/adjnoun.mtx"
#filename = "grafovi/ia-infect-hyper.mtx"
#filename = "grafovi/C125-9.mtx"
#filename = "grafovi/ia-enron-only.mtx"
#filename = "grafovi/c-fat200-1.mtx"
#filename = "grafovi/c-fat200-2.mtx"
#filename = "grafovi/c-fat200-5.mtx"
#filename = "grafovi/sphere.mtx"
#filename = "grafovi/DD244.mtx"
#filename = "grafovi/ca-netscience.mtx"
#filename = "grafovi/infect-dublin.mtx"
#filename = "grafovi/bio-diseasome.mtx"
#filename = "grafovi/web-polblogs.mtx"
#filename = "grafovi/DD687.mtx"
#filename = "grafovi/rt-twitter-copen.mtx"
#filename = "grafovi/DD68.mtx"
#filename = "grafovi/ia-crime-moreno.mtx"
#filename = "grafovi/DD199.mtx"
#filename = "grafovi/soc-wiki-Vote.mtx"
#filename = "grafovi/DD349.mtx"
#filename = "grafovi/DD497.mtx"
#filename = "grafovi/socfb-Reed98.mtx"
#filename = "grafovi/delaunay_n10.mtx"
#filename = "grafovi/email-univ.mtx"
#filename = "grafovi/econ-mahindas.mtx"
#filename = "grafovi/ia-fb-messages.mtx"
#filename = "grafovi/bio-yeast.mtx"
#filename = "grafovi/tech-routers-rf.mtx"
#filename = "grafovi/facebook.mtx"
#filename = "grafovi/squirrel.mtx"

#graph = load_graph_from_mtx(filename)


